# DSC 106 Midterm Project

In [106]:
#import packages
import spotipy
import spotipy.util as util
import pandas as pd
import urllib.request
import sklearn.svm as svm
from sklearn.linear_model import LogisticRegression
import numpy as np

In [47]:
from spotipy.oauth2 import SpotifyClientCredentials

cid = '327b0432454145edac87d743da39068e'
secret = 'b7e14bf491b4434ab70f1868fc1fba75'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

In [48]:
#Spotify API authentication token. The first parameter is the Spotify user id.
#token = util.prompt_for_user_token('biggurlandyouknowit',
#                                   client_id = '327b0432454145edac87d743da39068e',
#                                   client_secret = 'b7e14bf491b4434ab70f1868fc1fba75',
#                                   redirect_uri = 'http://localhost:8888/callback')
#access the Spotify API
#sp = spotipy.Spotify(auth=token)

In [186]:
#access the playlist we made that contains our training tracks
training_playlist = sp.user_playlist('biggurlandyouknowit', 'spotify:playlist:1ZouLC1keHDTVeXRQDx6dq')

In [187]:
#basic info for the training tracks
training_tracks_df = pd.DataFrame()
for track_data in training_playlist['tracks']['items']:
    track_df = pd.DataFrame(pd.DataFrame(dict([(k,pd.Series(v)) for k,v in track_data.items()]))['track']).transpose()
    training_tracks_df = training_tracks_df.append(track_df)
training_tracks_df = training_tracks_df.drop(training_tracks_df.columns[0], axis = 1).reset_index(drop = True)
training_tracks_df

C:\Users\richa\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
C:\Users\richa\Anaconda3\lib\site-packages\pandas\core\frame.py:7397: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes)


,external_urls,href,id,type,uri,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,is_local,name,popularity,preview_url,track,track_number,url
0,{'spotify': 'https://open.spotify.com/track/37...,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,37ZJ0p5Jm13JPevGcx4SkF,track,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,249293,False,False,{'isrc': 'USPR38619998'},False,Livin' On A Prayer,83,None,True,3,NaN
1,{'spotify': 'https://open.spotify.com/track/5H...,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,5HkFTCxSeJ3kGNyQJbT4rJ,track,spotify:track:5HkFTCxSeJ3kGNyQJbT4rJ,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,215933,False,False,{'isrc': 'GBCEE8000027'},False,Another One Bites The Dust,48,None,True,3,NaN
2,{'spotify': 'https://open.spotify.com/track/0G...,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,0G21yYKMZoHa30cYVi1iA8,track,spotify:track:0G21yYKMZoHa30cYVi1iA8,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,273480,False,False,{'isrc': 'USGF18714801'},False,Welcome To The Jungle,72,None,True,1,NaN
3,{'spotify': 'https://open.spotify.com/track/3f...,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,3fH4KjXFYMmljxrcGrbPj9,track,spotify:track:3fH4KjXFYMmljxrcGrbPj9,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, DK, DZ, FI, FR, GB, KW, MA, MX, NO, PL, P...",2,263040,False,False,{'isrc': 'GBAAA8500001'},False,Don't You (Forget About Me),75,None,True,1,NaN
4,{'spotify': 'https://open.spotify.com/track/6F...,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,6Fba9RZtC6vTY814JToDtP,track,spotify:track:6Fba9RZtC6vTY814JToDtP,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,239560,False,False,{'isrc': 'USWB18300001'},False,Jump,58,https://p.scdn.co/mp3-preview/5c4fca39cd964267...,True,7,NaN
5,{'spotify': 'https://open.spotify.com/track/6l...,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,6l8GvAyoUZwWDgF1e4822w,track,spotify:track:6l8GvAyoUZwWDgF1e4822w,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,355400,False,False,{'isrc': 'GBCEE0500364'},False,Bohemian Rhapsody,61,None,True,9,NaN
6,{'spotify': 'https://open.spotify.com/track/5M...,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,5MxNLUsfh7uzROypsoO5qe,track,spotify:track:5MxNLUsfh7uzROypsoO5qe,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,266960,False,False,{'isrc': 'USSM10011897'},False,Dream On,78,https://p.scdn.co/mp3-preview/4c7fea33f70ea106...,True,3,NaN
7,{'spotify': 'https://open.spotify.com/track/51...,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,51pQ7vY7WXzxskwloaeqyj,track,spotify:track:51pQ7vY7WXzxskwloaeqyj,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,478173,False,False,{'isrc': 'USAT29900620'},False,Stairway to Heaven - 1990 Remaster,69,https://p.scdn.co/mp3-preview/da57b9497b855355...,True,4,NaN
8,{'spotify': 'https://open.spotify.com/track/1l...,https://api.spotify.com/v1/tracks/1lbXEepatjRV...,1lbXEepatjRVjoG8pZMtdp,track,spotify:track:1lbXEepatjRVjoG8pZMtdp,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,254453,False,False,{'isrc': 'USWB19900178'},False,Dreams,62,https://p.scdn.co/mp3-preview/10b5068313e5ee28...,True,9,NaN
9,{'spotify': 'ht

In [188]:
#audio features for the training tracks
training_audio_features_df = pd.DataFrame()
for index, row in training_tracks_df.iterrows():
    audio_feature = pd.DataFrame(sp.audio_features('spotify:track:' + row['id']))
    training_audio_features_df = training_audio_features_df.append(audio_feature)
training_audio_features_df = training_audio_features_df.reset_index(drop = True)
training_audio_features_df.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.0778,https://api.spotify.com/v1/audio-analysis/37ZJ...,0.532,249293,0.887,37ZJ0p5Jm13JPevGcx4SkF,0.000214,0,0.2940,-3.757,1,0.0335,122.511,4,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,audio_features,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,0.795
1,0.0444,https://api.spotify.com/v1/audio-analysis/5HkF...,0.922,215933,0.392,5HkFTCxSeJ3kGNyQJbT4rJ,0.165000,5,0.1230,-12.083,0,0.1220,109.878,4,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,audio_features,spotify:track:5HkFTCxSeJ3kGNyQJbT4rJ,0.771
2,0.0195,https://api.spotify.com/v1/audio-analysis/0G21...,0.453,273480,0.987,0G21yYKMZoHa30cYVi1iA8,0.283000,8,0.2680,-4.485,1,0.0889,123.544,4,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,audio_features,spotify:track:0G21yYKMZoHa30cYVi1iA8,0.316
3,0.1680,https://api.spotify.com/v1/audio-analysis/3fH4...,0.660,263040,0.816,3fH4KjXFYMmljxrcGrbPj9,0.018100,2,0.0608,-6.610,1,0.0299,111.346,4,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,audio_features,spotify:track:3fH4KjXFYMmljxrcGrbPj9,0.678
4,0.0745,https://api.spotify.com/v1/audio-analysis/6Fba...,0.547,239560,0.822,6Fba9RZtC6vTY814JToDtP,0.009430,0,0.0645,-7.201,1,0.0292,130.138,4,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,audio_features,spotify:track:6Fba9RZtC6vTY814JToDtP,0.774


In [189]:
#training tracks with BOTH basic track info and audio features combined
training_df = training_tracks_df.merge(training_audio_features_df, on = 'id')
training_df

,external_urls,href,id,type_x,uri_x,album,artists,available_markets,disc_number,duration_ms_x,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type_y,uri_y,valence
0,{'spotify': 'https://open.spotify.com/track/37...,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,37ZJ0p5Jm13JPevGcx4SkF,track,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,249293,...,0.2940,-3.757,1,0.0335,122.511,4,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,audio_features,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,0.795
1,{'spotify': 'https://open.spotify.com/track/5H...,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,5HkFTCxSeJ3kGNyQJbT4rJ,track,spotify:track:5HkFTCxSeJ3kGNyQJbT4rJ,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,215933,...,0.1230,-12.083,0,0.1220,109.878,4,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,audio_features,spotify:track:5HkFTCxSeJ3kGNyQJbT4rJ,0.771
2,{'spotify': 'https://open.spotify.com/track/0G...,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,0G21yYKMZoHa30cYVi1iA8,track,spotify:track:0G21yYKMZoHa30cYVi1iA8,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,273480,...,0.2680,-4.485,1,0.0889,123.544,4,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,audio_features,spotify:track:0G21yYKMZoHa30cYVi1iA8,0.316
3,{'spotify': 'https://open.spotify.com/track/3f...,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,3fH4KjXFYMmljxrcGrbPj9,track,spotify:track:3fH4KjXFYMmljxrcGrbPj9,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, DK, DZ, FI, FR, GB, KW, MA, MX, NO, PL, P...",2,263040,...,0.0608,-6.610,1,0.0299,111.346,4,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,audio_features,spotify:track:3fH4KjXFYMmljxrcGrbPj9,0.678
4,{'spotify': 'https://open.spotify.com/track/6F...,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,6Fba9RZtC6vTY814JToDtP,track,spotify:track:6Fba9RZtC6vTY814JToDtP,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,239560,...,0.0645,-7.201,1,0.0292,130.138,4,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,audio_features,spotify:track:6Fba9RZtC6vTY814JToDtP,0.774
5,{'spotify': 'https://open.spotify.com/track/6l...,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,6l8GvAyoUZwWDgF1e4822w,track,spotify:track:6l8GvAyoUZwWDgF1e4822w,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,355400,...,0.1880,-10.405,0,0.0503,144.242,4,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,audio_features,spotify:track:6l8GvAyoUZwWDgF1e4822w,0.210
6,{'spotify': 'https://open.spotify.com/track/5M...,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,5MxNLUsfh7uzROypsoO5qe,track,spotify:track:5MxNLUsfh7uzROypsoO5qe,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,266960,...,0.3320,-10.057,1,0.0290,160.900,4,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,audio_features,spotify:track:5MxNLUsfh7uzROypsoO5qe,0.224
7,{'spotify': 'https://open.spotify.com/track/51...,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,51pQ7vY7WXzxskwloaeqyj,track,spotify:track:51pQ7vY7WXzxskwloaeqyj,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,478173,...,0.1850,-12.453,1,0.0339,84.204,4,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,audio_features,spotify:track:51pQ7vY7WXzxskwloaeqyj,0.213
8,{'spotify': 'https://open.spotify.com/track/1l...,https://api.spotify.com/v1/tracks

In [190]:
#extract release date as a column
def getReleaseDate(x):
    return x['album']['release_date']
training_df['release_date'] = training_df.apply(getReleaseDate, axis = 1)

In [191]:
#create a decade column
def make_decade(x):
    year = int(x[0:4])
    if year >= 1950 and year < 1960:
        return 0
    if year >= 1960 and year < 1970:
        return 1
    if year >= 1970 and year < 1980:
        return 2
    if year >= 1980 and year < 1990:
        return 3
    if year >= 1990 and year < 2000:
        return 4
    if year >= 2000 and year < 2010:
        return 5
    if year >= 2010 and year < 2020:
        return 6
    if year >= 2020 and year < 2030:
        return 7
training_df['decade'] = training_df['release_date'].apply(make_decade)

In [192]:
training_df = training_df.drop(['episode','is_local','popularity','duration_ms_y', 'external_ids', 'track_number', 'track', 'track_href', 'key', 'type_y', 'uri_y', 'external_urls', 'href', 'id', 'type_x', 'uri_x', 'album', 'artists', 'available_markets', 'disc_number'], axis = 1)

In [193]:
training_songs = training_df[['valence', 'energy', 'danceability', 'loudness', 'speechiness', 'tempo', 'instrumentalness']]
training_songs.head()

,valence,energy,danceability,loudness,speechiness,tempo,instrumentalness
0,0.795,0.887,0.532,-3.757,0.0335,122.511,0.000214
1,0.771,0.392,0.922,-12.083,0.1220,109.878,0.165000
2,0.316,0.987,0.453,-4.485,0.0889,123.544,0.283000
3,0.678,0.816,0.660,-6.610,0.0299,111.346,0.018100
4,0.774,0.822,0.547,-7.201,0.0292,130.138,0.009430


In [195]:
train_decade = [3,3,3,3,3,2,2,2,2,2,0,0,0,0,0,1,1,1,1,1,4,4,4,4,4,5,5,5,5,6,6,6,6,6,5]

In [206]:
classifier = LogisticRegression(C = 5)

In [207]:
#classfier based on valence, energy, danceability, loudness, speechiness, tempo, instrumentalness
classifier.fit(training_songs, train_decade)

C:\Users\richa\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [208]:
sol = classifier.predict(training_songs)

In [209]:
np.mean(train_decade == sol)

0.5714285714285714

In [210]:
#access the playlist we made that contains our testing tracks
test_playlist = sp.user_playlist('biggurlandyouknowit', 'spotify:playlist:6hvbipGQk0VYCjr5oXORBh')

In [168]:
#basic info for the training tracks
test_tracks_df = pd.DataFrame()
for track_data in test_playlist['tracks']['items']:
    track_df = pd.DataFrame(pd.DataFrame(dict([(k,pd.Series(v)) for k,v in track_data.items()]))['track']).transpose()
    test_tracks_df = test_tracks_df.append(track_df)
test_tracks_df = test_tracks_df.drop(test_tracks_df.columns[0], axis = 1).reset_index(drop = True)
test_tracks_df

C:\Users\richa\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
C:\Users\richa\Anaconda3\lib\site-packages\pandas\core\frame.py:7397: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes)


,external_urls,href,id,type,uri,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,is_local,name,popularity,preview_url,track,track_number,url
0,{'spotify': 'https://open.spotify.com/track/2m...,https://api.spotify.com/v1/tracks/2m1hi0nfMR9v...,2m1hi0nfMR9vdGC8UcrnwU,track,spotify:track:2m1hi0nfMR9vdGC8UcrnwU,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,167066,False,False,{'isrc': 'USMC19959123'},False,All The Small Things,79,None,True,8,NaN
1,{'spotify': 'https://open.spotify.com/track/5g...,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,5ghIJDpPoe3CfHMGu71E6T,track,spotify:track:5ghIJDpPoe3CfHMGu71E6T,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, MX, US]",1,301920,False,False,{'isrc': 'USGF19942501'},False,Smells Like Teen Spirit,74,None,True,1,NaN
2,{'spotify': 'https://open.spotify.com/track/5j...,https://api.spotify.com/v1/tracks/5jafMI8FLibn...,5jafMI8FLibnjkYTZ33m0c,track,spotify:track:5jafMI8FLibnjkYTZ33m0c,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,257480,False,False,{'isrc': 'GBAYE9400055'},False,High And Dry,73,https://p.scdn.co/mp3-preview/7cc3982631523940...,True,3,NaN
3,{'spotify': 'https://open.spotify.com/track/5l...,https://api.spotify.com/v1/tracks/5lWRaa0fBxDE...,5lWRaa0fBxDE5yU91npPq7,track,spotify:track:5lWRaa0fBxDE5yU91npPq7,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, MX, US]",1,209200,False,False,{'isrc': 'USIR19500272'},False,Just A Girl,64,None,True,3,NaN
4,{'spotify': 'https://open.spotify.com/track/6h...,https://api.spotify.com/v1/tracks/6hTcuIQa0sxr...,6hTcuIQa0sxrrByu9wTD7s,track,spotify:track:6hTcuIQa0sxrrByu9wTD7s,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,269920,False,False,{'isrc': 'USSM17500423'},False,Born to Run,74,https://p.scdn.co/mp3-preview/bac7b351bb2f85c7...,True,5,NaN
5,{'spotify': 'https://open.spotify.com/track/2n...,https://api.spotify.com/v1/tracks/2nVHqZbOGkKW...,2nVHqZbOGkKWzlcy1aMbE7,track,spotify:track:2nVHqZbOGkKWzlcy1aMbE7,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,508200,False,False,{'isrc': 'USAT29900621'},False,Kashmir - 1990 Remaster,70,https://p.scdn.co/mp3-preview/13b2c47c98132f80...,True,6,NaN
6,{'spotify': 'https://open.spotify.com/track/4k...,https://api.spotify.com/v1/tracks/4kzvAGJirpZ9...,4kzvAGJirpZ9ethvKZdJtg,track,spotify:track:4kzvAGJirpZ9ethvKZdJtg,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,180666,False,False,{'isrc': 'GBCEE7700016'},False,We Are The Champions,52,None,True,2,NaN
7,{'spotify': 'https://open.spotify.com/track/5W...,https://api.spotify.com/v1/tracks/5Wj1rJnCLpMH...,5Wj1rJnCLpMHdLaxsFtJLs,track,spotify:track:5Wj1rJnCLpMHdLaxsFtJLs,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,322548,False,False,{'isrc': 'GBUM71304955'},False,Bennie And The Jets - Remastered 2014,74,None,True,3,NaN
8,{'spotify': 'https://open.spotify.com/track/2y...,https://api.spotify.com/v1/tracks/2yBVeksU2Etr...,2yBVeksU2EtrPJbTu4ZslK,track,spotify:track:2yBVeksU2EtrPJbTu4ZslK,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,223866,False,False,{'isrc': 'USWB10902441'},False,What a Fool Believes,70,https://p.scdn.co/mp3-preview/38dcede79a2e68d9...,True,2,NaN
9,{'spotify': 'https://open.spotif

In [169]:
#audio features for the training tracks
test_audio_features_df = pd.DataFrame()
for index, row in test_tracks_df.iterrows():
    audio_feature = pd.DataFrame(sp.audio_features('spotify:track:' + row['id']))
    test_audio_features_df = test_audio_features_df.append(audio_feature)
test_audio_features_df = test_audio_features_df.reset_index(drop = True)
test_audio_features_df.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.010300,https://api.spotify.com/v1/audio-analysis/2m1h...,0.434,167067,0.897,2m1hi0nfMR9vdGC8UcrnwU,0.000000,0,0.6120,-4.918,1,0.0488,148.726,4,https://api.spotify.com/v1/tracks/2m1hi0nfMR9v...,audio_features,spotify:track:2m1hi0nfMR9vdGC8UcrnwU,0.684
1,0.000025,https://api.spotify.com/v1/audio-analysis/5ghI...,0.502,301920,0.912,5ghIJDpPoe3CfHMGu71E6T,0.000173,1,0.1060,-4.556,1,0.0564,116.761,4,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,audio_features,spotify:track:5ghIJDpPoe3CfHMGu71E6T,0.720
2,0.071800,https://api.spotify.com/v1/audio-analysis/5jaf...,0.418,257480,0.383,5jafMI8FLibnjkYTZ33m0c,0.017700,4,0.0896,-11.782,1,0.0257,87.773,4,https://api.spotify.com/v1/tracks/5jafMI8FLibn...,audio_features,spotify:track:5jafMI8FLibnjkYTZ33m0c,0.352
3,0.066200,https://api.spotify.com/v1/audio-analysis/5lWR...,0.638,209200,0.906,5lWRaa0fBxDE5yU91npPq7,0.000069,2,0.1420,-4.824,1,0.0504,107.991,4,https://api.spotify.com/v1/tracks/5lWRaa0fBxDE...,audio_features,spotify:track:5lWRaa0fBxDE5yU91npPq7,0.767
4,0.005010,https://api.spotify.com/v1/audio-analysis/6hTc...,0.270,269920,0.944,6hTcuIQa0sxrrByu9wTD7s,0.000021,4,0.1160,-4.199,1,0.0975,146.347,4,https://api.spotify.com/v1/tracks/6hTcuIQa0sxr...,audio_features,spotify:track:6hTcuIQa0sxrrByu9wTD7s,0.606


In [170]:
#training tracks with BOTH basic track info and audio features combined
test_df = test_tracks_df.merge(test_audio_features_df, on = 'id')
test_df

,external_urls,href,id,type_x,uri_x,album,artists,available_markets,disc_number,duration_ms_x,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type_y,uri_y,valence
0,{'spotify': 'https://open.spotify.com/track/2m...,https://api.spotify.com/v1/tracks/2m1hi0nfMR9v...,2m1hi0nfMR9vdGC8UcrnwU,track,spotify:track:2m1hi0nfMR9vdGC8UcrnwU,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,167066,...,0.6120,-4.918,1,0.0488,148.726,4,https://api.spotify.com/v1/tracks/2m1hi0nfMR9v...,audio_features,spotify:track:2m1hi0nfMR9vdGC8UcrnwU,0.684
1,{'spotify': 'https://open.spotify.com/track/5g...,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,5ghIJDpPoe3CfHMGu71E6T,track,spotify:track:5ghIJDpPoe3CfHMGu71E6T,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, MX, US]",1,301920,...,0.1060,-4.556,1,0.0564,116.761,4,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,audio_features,spotify:track:5ghIJDpPoe3CfHMGu71E6T,0.720
2,{'spotify': 'https://open.spotify.com/track/5j...,https://api.spotify.com/v1/tracks/5jafMI8FLibn...,5jafMI8FLibnjkYTZ33m0c,track,spotify:track:5jafMI8FLibnjkYTZ33m0c,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,257480,...,0.0896,-11.782,1,0.0257,87.773,4,https://api.spotify.com/v1/tracks/5jafMI8FLibn...,audio_features,spotify:track:5jafMI8FLibnjkYTZ33m0c,0.352
3,{'spotify': 'https://open.spotify.com/track/5l...,https://api.spotify.com/v1/tracks/5lWRaa0fBxDE...,5lWRaa0fBxDE5yU91npPq7,track,spotify:track:5lWRaa0fBxDE5yU91npPq7,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, MX, US]",1,209200,...,0.1420,-4.824,1,0.0504,107.991,4,https://api.spotify.com/v1/tracks/5lWRaa0fBxDE...,audio_features,spotify:track:5lWRaa0fBxDE5yU91npPq7,0.767
4,{'spotify': 'https://open.spotify.com/track/6h...,https://api.spotify.com/v1/tracks/6hTcuIQa0sxr...,6hTcuIQa0sxrrByu9wTD7s,track,spotify:track:6hTcuIQa0sxrrByu9wTD7s,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,269920,...,0.1160,-4.199,1,0.0975,146.347,4,https://api.spotify.com/v1/tracks/6hTcuIQa0sxr...,audio_features,spotify:track:6hTcuIQa0sxrrByu9wTD7s,0.606
5,{'spotify': 'https://open.spotify.com/track/2n...,https://api.spotify.com/v1/tracks/2nVHqZbOGkKW...,2nVHqZbOGkKWzlcy1aMbE7,track,spotify:track:2nVHqZbOGkKWzlcy1aMbE7,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,508200,...,0.1550,-11.166,1,0.0427,80.617,3,https://api.spotify.com/v1/tracks/2nVHqZbOGkKW...,audio_features,spotify:track:2nVHqZbOGkKWzlcy1aMbE7,0.581
6,{'spotify': 'https://open.spotify.com/track/4k...,https://api.spotify.com/v1/tracks/4kzvAGJirpZ9...,4kzvAGJirpZ9ethvKZdJtg,track,spotify:track:4kzvAGJirpZ9ethvKZdJtg,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,180666,...,0.1010,-7.845,1,0.0298,94.504,3,https://api.spotify.com/v1/tracks/4kzvAGJirpZ9...,audio_features,spotify:track:4kzvAGJirpZ9ethvKZdJtg,0.189
7,{'spotify': 'https://open.spotify.com/track/5W...,https://api.spotify.com/v1/tracks/5Wj1rJnCLpMH...,5Wj1rJnCLpMHdLaxsFtJLs,track,spotify:track:5Wj1rJnCLpMHdLaxsFtJLs,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,322548,...,0.4970,-7.950,1,0.0493,132.642,4,https://api.spotify.com/v1/tracks/5Wj1rJnCLpMH...,audio_features,spotify:track:5Wj1rJnCLpMHdLaxsFtJLs,0.779
8,{'spotify': 'https://open.spotify.com/track/2y...,https://api.spotify.com/v1/tracks/2yBVeksU2Etr...,2yBVeksU2EtrPJbTu4Zs

In [171]:
test_df['release_date'] = test_df.apply(getReleaseDate, axis = 1)

In [172]:
test_df['decade'] = test_df['release_date'].apply(make_decade)

In [173]:
test_df = test_df.drop(['episode','is_local','popularity','duration_ms_y', 'external_ids', 'track_number', 'track', 'track_href', 'key', 'type_y', 'uri_y', 'external_urls', 'href', 'id', 'type_x', 'uri_x', 'album', 'artists', 'available_markets', 'disc_number'], axis = 1)

In [174]:
test_songs = test_df[['valence', 'energy', 'danceability', 'loudness', 'speechiness', 'tempo', 'instrumentalness']]
test_songs.head()

,valence,energy,danceability,loudness,speechiness,tempo,instrumentalness
0,0.684,0.897,0.434,-4.918,0.0488,148.726,0.000000
1,0.720,0.912,0.502,-4.556,0.0564,116.761,0.000173
2,0.352,0.383,0.418,-11.782,0.0257,87.773,0.017700
3,0.767,0.906,0.638,-4.824,0.0504,107.991,0.000069
4,0.606,0.944,0.270,-4.199,0.0975,146.347,0.000021


In [213]:
test_decade = [4,4,4,4,2,2,2,2,2,1,1,1,1,1,3,3,3,3,3,5,5,5,5,5,6,6,6,6,6,0,0,0,0,0,0]

In [214]:
test_sol = classifier.predict(test_songs)

In [215]:
np.mean(test_decade == test_sol)

0.34285714285714286

In [216]:
test_sol

array([5, 6, 2, 3, 5, 2, 2, 3, 2, 3, 1, 1, 0, 0, 6, 1, 3, 3, 1, 5, 5, 6,
       2, 3, 3, 3, 3, 3, 3, 0, 0, 5, 1, 0, 3])

In [184]:
np.array(test_decade)

array([4, 4, 4, 4, 2, 2, 2, 2, 2, 1, 1, 3, 6, 6, 3, 3, 3, 4, 3, 5, 5, 5,
       5, 5, 6, 6, 6, 6, 6, 4, 6, 0, 0, 6, 0], dtype=int64)

In [212]:
test_df.head()

,duration_ms_x,explicit,name,preview_url,url,acousticness,analysis_url,danceability,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,time_signature,valence,release_date,decade
0,167066,False,All The Small Things,None,NaN,0.010300,https://api.spotify.com/v1/audio-analysis/2m1h...,0.434,0.897,0.000000,0.6120,-4.918,1,0.0488,148.726,4,0.684,1999-06-01,4
1,301920,False,Smells Like Teen Spirit,None,NaN,0.000025,https://api.spotify.com/v1/audio-analysis/5ghI...,0.502,0.912,0.000173,0.1060,-4.556,1,0.0564,116.761,4,0.720,1991-09-26,4
2,257480,False,High And Dry,https://p.scdn.co/mp3-preview/7cc3982631523940...,NaN,0.071800,https://api.spotify.com/v1/audio-analysis/5jaf...,0.418,0.383,0.017700,0.0896,-11.782,1,0.0257,87.773,4,0.352,1995-03-28,4
3,209200,False,Just A Girl,None,NaN,0.066200,https://api.spotify.com/v1/audio-analysis/5lWR...,0.638,0.906,0.000069,0.1420,-4.824,1,0.0504,107.991,4,0.767,1995-10-10,4
4,269920,False,Born to Run,https://p.scdn.co/mp3-preview/bac7b351bb2f85c7...,NaN,0.005010,https://api.spotify.com/v1/audio-analysis/6hTc...,0.270,0.944,0.000021,0.1160,-4.199,1,0.0975,146.347,4,0.606,1975-08-25,2
